In [1]:
# loads model + val/test set, evaluates accuracy

import torch
from model import build_model_r3d_18  # replace with actual class name
from dataset import ClipDataset
from torch.utils.data import DataLoader
import pandas as pd


In [6]:
# ========== SETUP ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load test dataset
test_csv_path = "/home/atupulazi/personal_projects/collision-detection/frames/test/test_clip_labels.csv"  # <-- update if different
test_dataset = ClipDataset(test_csv_path, split='test')
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [7]:
# Load model
model = build_model_r3d_18()  # Replace with your model class
model.load_state_dict(torch.load("/home/atupulazi/personal_projects/collision-detection/src/checkpoints/r3d18_final.pth"))  # Your .pth file here
model.to(device)
model.eval()


VideoResNet(
  (stem): BasicStem(
    (0): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (conv2): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1):

In [8]:
# ========== TEST LOOP ==========
correct = 0
total = 0

with torch.no_grad():
    for clips, labels in test_loader:
        clips, labels = clips.to(device), labels.to(device)
        outputs = model(clips)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 73.75%
